The notebook provides a basic example of accessing the SNT API through pyimagej.

In [1]:
import sys
#!conda install --yes --prefix {sys.prefix} -c conda-forge pyimagej openjdk=8

In [2]:
import os
import imagej

In [3]:
# Initialize Fiji with GUI support. Use headless=True to run in headless mode.
ij = imagej.init(r'C:\Users\cam\Desktop\Fiji.app', headless=False)
# Now that Fiji is initialized we can import jnius. Note this must be done AFTER Fiji is initialized.
from jnius import autoclass, cast

As would be done in a Jython script using Fiji's Script Editor, we need to import the SNT classes we want to use.
The ```jnius.autoclass``` function allows us to do exactly that, simply pass the full classpath as an argument to the function.

The most up-to-date API is found at https://morphonets.github.io/SNT/

For example, to download a neuron reconstruction from the MouseLight database and calculate summary statistics on it, you would import the MouseLightLoader and TreeStatistics classes, as shown in the following cell.

In [4]:
# https://morphonets.github.io/SNT/sc/fiji/snt/io/MouseLightLoader.html
MouseLightLoader = autoclass('sc.fiji.snt.io.MouseLightLoader')
# https://morphonets.github.io/SNT/sc/fiji/snt/analysis/TreeAnalyzer.html
TreeStatistics = autoclass('sc.fiji.snt.analysis.TreeStatistics')

Now you can access all the attributes and methods these classes offer. 

Let's get a summary of the inter-node distances for a specific mouse cortical motor neuron (ID = "AA0100" in the [MouseLight database](http://ml-neuronbrowser.janelia.org/)).

In [5]:
def run():
    # First, initialize MouseLightLoader with the ID String of the desired reconstruction.
    loader = MouseLightLoader("AA0100")
    if not loader.isDatabaseAvailable():
        print("Could not connect to ML database", "Error")
        return
    if not loader.idExists():
        print("Somehow the specified id was not found", "Error")
        return

    # Specific neurite compartments may be extracted from the loader instance.
    a_tree = loader.getTree('axon', None) # compartment, color
    # Construct a TreeStatistics instance using the axon subtree.
    s_stats = TreeStatistics(a_tree)
    # As is customary in Java, constant fields are named using capital leters, with words separated by underscores.
    # The TreeStatistics class provides a number of constants which may be passed to any of its methods.
    metric = TreeStatistics.INTER_NODE_DISTANCE
    # the getSummaryStats() method returns a SummaryStatistics instance, whose methods are detailed here
    # https://commons.apache.org/proper/commons-math/javadocs/api-3.3/org/apache/commons/math3/stat/descriptive/SummaryStatistics.html
    summary_stats = s_stats.getSummaryStats(metric)
    print("The average inter-node distance is %d" % summary_stats.getMean())
    # To plot a histogram of the metric, do
    s_stats.getHistogram(metric).show()


In [6]:
run()

The average inter-node distance is 21
